# Model training using trainers
This use-case is model training - the same, but now the usage of Trainer will be shown.

In [1]:
#!pip3 install torchvision

In [2]:
import cascade.data as cdd
import cascade.models as cdm
from cascade.utils.torch_model import TorchModel

from tqdm import tqdm
import torch
import torchvision
from torchvision.transforms import functional as F
from torch import nn
from sklearn.metrics import accuracy_score

In [3]:
import cascade
cascade.__version__

'0.11.0'

## Defining data pipeline
This part will be without comments. For more detailed explanations, please see [pipeline building example](https://oxid15.github.io/cascade/examples/pipeline_building.html)

In [4]:
MNIST_ROOT = 'data'
INPUT_SIZE = 784
BATCH_SIZE = 10

In [5]:
class NoiseModifier(cdd.Modifier):
    def __getitem__(self, index):
        img, label = self._dataset[index]
        img += torch.rand_like(img) * 0.1
        img = torch.clip(img, 0, 255)
        return img, label


train_ds = torchvision.datasets.MNIST(root=MNIST_ROOT,
                                     train=True, 
                                     transform=F.to_tensor,
                                     download=True)
test_ds = torchvision.datasets.MNIST(root=MNIST_ROOT, 
                                    train=False, 
                                    transform=F.to_tensor)

train_ds = cdd.Wrapper(train_ds, 
    meta_prefix={
        'desc': 'This is MNIST dataset of handwritten images, TRAIN PART'
    })
test_ds = cdd.Wrapper(test_ds)

train_ds = NoiseModifier(train_ds)
test_ds = NoiseModifier(test_ds)

# We will constraint the number of samples to speed up learning in example
train_ds = cdd.CyclicSampler(train_ds, 10000)
test_ds = cdd.CyclicSampler(test_ds, 5000)

train_dl = torch.utils.data.DataLoader(dataset=train_ds, 
                                       batch_size=BATCH_SIZE,
                                       shuffle=True)
test_dl = torch.utils.data.DataLoader(dataset=test_ds,
                                      batch_size=BATCH_SIZE,
                                      shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [6]:
train_ds.get_meta()

[{'name': 'cascade.data.cyclic_sampler.CyclicSampler',
  'type': 'dataset',
  'len': 10000},
 {'name': '__main__.NoiseModifier', 'type': 'dataset', 'len': 60000},
 {'name': 'cascade.data.dataset.Wrapper',
  'desc': 'This is MNIST dataset of handwritten images, TRAIN PART',
  'type': 'dataset',
  'len': 60000,
  'obj_type': "<class 'torchvision.datasets.mnist.MNIST'>"}]

## Model definition
Before training we need to define our model. We need regular nn.Module and Cascade's wrapper around it.  
  
Module defined without any specific changes in the original pytorch code, except now it accepts `*args` and `**kwargs` in `__init__`

In [7]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, *args, **kwargs):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, y):
         out = self.l1(y)
         out = self.relu(out)
         out = self.l2(out)

         return out

Next Cascade's wrapper is defined. The most of the interaction with pytorch modules are already implemented in `cascade.utils.TorchModel` so we need to only define how to train and evaluate the model.  
  
The difference between previous example and this in the `fit` function - now it only fits one epoch per call and doesn't need additional logging - Trainer will cover this functionality.

In [8]:
class Classifier(TorchModel):
    # In train we copy-paste regular pytorch trainloop, 
    # but use self._model, where our SimpleNN is placed
    def fit(self, train_dl, lr, *args, **kwargs):
        criterion = nn.CrossEntropyLoss()
        optim = torch.optim.Adam(self._model.parameters(), lr=lr)

        ds_size = len(train_dl)
        for x, (imgs, labels) in enumerate(train_dl): 
            imgs = imgs.reshape(-1, self._model.input_size)

            out = self._model(imgs)
            loss = criterion(out, labels)

            optim.zero_grad()
            loss.backward()
            optim.step() 


    # Evaluate function takes the metrics from arguments
    # and populates self.metrics without returning anything
    def evaluate(self, test_dl, metrics_dict, *args, **kwargs):
        pred = []
        gt = []
        for imgs, labels in tqdm(test_dl): 
            imgs = imgs.reshape(-1, self._model.input_size)
            out = torch.argmax(self._model(imgs, *args, **kwargs), -1)

            pred.append(out)
            gt.append(labels)

        pred = torch.concat(pred).detach().numpy()
        gt = torch.concat(gt).detach().numpy()

        for metric_name in metrics_dict:
            self.metrics[metric_name] = metrics_dict[metric_name](gt, pred)

### Model initialization

In [9]:
NUM_EPOCHS = 10
LR = 1e-3

# Classifier will initialize SimpleNN with all the parameters passed
# but some of them are not for the SimpleNN, but to be recorded in metadata
model = Classifier(SimpleNN,
    # These arguments are needed by SimpleNN, 
    # but passed as keywords to be recorded in meta
    input_size=INPUT_SIZE,
    hidden_size=100,
    num_classes=10,
    # These arguments will be skipped by SimpleNN,
    # but will be added to meta
    num_epochs=NUM_EPOCHS,
    lr=LR,
    bs=BATCH_SIZE)

## Set up trainer
Let's set up logging first to catch trainer's logs

In [10]:
import sys
import logging
logging.basicConfig(
    handlers=[logging.StreamHandler(sys.stdout)],
    level='INFO'
)

In [11]:
# Trainer accepts ModelRepo object or just a path 
trainer = cdm.BasicTrainer('trainer_repo')

In [11]:
# The main method of course is train
# It will do all the stuff needed for us
# including training, evaluating, saving and logging
trainer.train(
    model,
    train_data=train_dl,
    test_data=test_dl,
    train_kwargs={'lr': LR, 'bs': BATCH_SIZE}, # will be passed into model.fit()
    test_kwargs={'metrics_dict': {'acc': accuracy_score}}, # will be passed into model.evaluate()
    epochs=NUM_EPOCHS,
    start_from=None, # can start from checkpoint if line is specified,
    save_strategy=2,
    eval_strategy=1
)

INFO:cascade.models.trainer:Training started with parameters:
{'lr': 0.001, 'bs': 10}
INFO:cascade.models.trainer:repo is ModelRepo in trainer_repo of 1 lines
INFO:cascade.models.trainer:line is 00000
INFO:cascade.models.trainer:training will last 10 epochs


100%|██████████| 500/500 [00:00<00:00, 836.31it/s]

INFO:cascade.models.trainer:Epoch 0: {'acc': 0.874}



100%|██████████| 500/500 [00:00<00:00, 827.84it/s]


INFO:cascade.models.trainer:Epoch 1: {'acc': 0.8978}


...

INFO:cascade.models.trainer:Epoch 7: {'acc': 0.9328}



100%|██████████| 500/500 [00:00<00:00, 835.19it/s]

INFO:cascade.models.trainer:Epoch 8: {'acc': 0.9294}



100%|██████████| 500/500 [00:00<00:00, 871.34it/s]

INFO:cascade.models.trainer:Epoch 9: {'acc': 0.9282}
INFO:cascade.models.trainer:Training finished in 22 seconds


## Results
We can obtain the results of training from trainer's meta data.

In [12]:
trainer.get_meta()

[{'name': '<cascade.models.trainer.BasicTrainer object at 0x7f820f40beb0>',
  'train_start_at': DateTime(2023, 3, 30, 10, 44, 51, 579370, tzinfo=Timezone('Europe/Moscow')),
  'train_end_at': DateTime(2023, 3, 30, 10, 45, 14, 547613, tzinfo=Timezone('Europe/Moscow')),
  'metrics': [{'acc': 0.874},
   {'acc': 0.8978},
   {'acc': 0.9092},
   {'acc': 0.9258},
   {'acc': 0.919},
   {'acc': 0.9164},
   {'acc': 0.9268},
   {'acc': 0.9328},
   {'acc': 0.9294},
   {'acc': 0.9282}],
  'repo': [{'name': 'ModelRepo in trainer_repo of 1 lines',
    'root': 'trainer_repo',
    'len': 1,
    'updated_at': DateTime(2023, 3, 30, 7, 45, 14, 570552, tzinfo=Timezone('UTC')),
    'type': 'repo'}]}]

## Start from checkpoint
Let's try continue learning where we finished using the same line as before.

In [13]:
trainer.train(
    model,
    train_data=train_dl,
    test_data=test_dl,
    train_kwargs={'lr': LR, 'bs': BATCH_SIZE},
    test_kwargs={'metrics_dict': {'acc': accuracy_score}},
    epochs=5,
    start_from='00000',
    save_strategy=4,
    eval_strategy=1
)

[Errno 2] No such file or directory: 'trainer_repo/00000/00009/model'
INFO:cascade.models.trainer:Training started with parameters:
{'lr': 0.001, 'bs': 10}
INFO:cascade.models.trainer:repo is ModelRepo in trainer_repo of 1 lines
INFO:cascade.models.trainer:line is 00000
INFO:cascade.models.trainer:started from model None
INFO:cascade.models.trainer:training will last 5 epochs


100%|██████████| 500/500 [00:00<00:00, 851.19it/s]

INFO:cascade.models.trainer:Epoch 0: {'acc': 0.9396}



100%|██████████| 500/500 [00:00<00:00, 850.32it/s]

INFO:cascade.models.trainer:Epoch 1: {'acc': 0.9384}


...

INFO:cascade.models.trainer:Epoch 2: {'acc': 0.9378}



100%|██████████| 500/500 [00:00<00:00, 858.31it/s]

INFO:cascade.models.trainer:Epoch 3: {'acc': 0.941}



100%|██████████| 500/500 [00:00<00:00, 878.11it/s]

INFO:cascade.models.trainer:Epoch 4: {'acc': 0.9162}
INFO:cascade.models.trainer:Training finished in 11 seconds


In [14]:
trainer.metrics

[{'acc': 0.874},
 {'acc': 0.8978},
 {'acc': 0.9092},
 {'acc': 0.9258},
 {'acc': 0.919},
 {'acc': 0.9164},
 {'acc': 0.9268},
 {'acc': 0.9328},
 {'acc': 0.9294},
 {'acc': 0.9282},
 {'acc': 0.9396},
 {'acc': 0.9384},
 {'acc': 0.9378},
 {'acc': 0.941},
 {'acc': 0.9162}]

## See also:
- [Pipeline building](pipeline_building.html)